In [12]:
import os 
import sys 
from typing import List 

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

from functools import cache 
import numpy as np 
import pandas as pd 
import altair as alt 
from IPython.display import clear_output
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

from utils_notebook.utils import ddf, load_subgraph, remove_prefix
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    stack_order_expr, 
    wide_to_longwide, 
    chart, 
)
from utils_notebook.queries import adjust_precision, QueryManager
from utils_notebook.testing import validate_season_series
from utils_notebook.css import css_tooltip_timeseries_multi_colored

In [13]:
sg: Subgrounds
bs: Subgraph
sg, bs = load_subgraph()

In [14]:
q = QueryManager(sg, bs) 

In [15]:
df = q.query_silo_daily_snapshots()

In [16]:
df_szns = q.query_seasons()

In [17]:
df = df.merge(df_szns, how='left', on='season')
df = df.drop(columns=['season'])
df.head()

,dailyBeanMints,timestamp
0,31.650670,2021-08-07 01:07:38
1,17.760260,2021-08-07 02:09:28
2,43.709604,2021-08-07 03:07:35
3,0.342173,2021-08-07 16:16:35
4,676.195254,2021-08-07 19:12:47


In [18]:
df = df.resample("W", on="timestamp").sum().reset_index()
df = df.rename(columns=dict(dailyBeanMints='weekly_silo_emissions'))
df['total_silo_emissions'] = df.weekly_silo_emissions.cumsum()
df.head()

,timestamp,weekly_silo_emissions,total_silo_emissions
0,2021-08-08,21192.194431,2.119219e+04
1,2021-08-15,238628.922359,2.598211e+05
2,2021-08-22,278865.602534,5.386867e+05
3,2021-08-29,312848.021805,8.515347e+05
4,2021-09-05,329166.748664,1.180701e+06


In [19]:
c = chart(
    wide_to_longwide(df, 'timestamp', ['timestamp'], ['total_silo_emissions', 'weekly_silo_emissions']), 
    'timestamp', 
    ['weekly_silo_emissions'],
    rmetrics=['total_silo_emissions'], 
    lstrategy='stack_bar', 
    rstrategy='line', 
    title="Silo Emissions (Weekly)", 
    yaxis_left_kwargs=dict(title="Beans", format=".3~s"), 
    yaxis_right_kwargs=dict(title="Beans", format=".3~s"), 
    dual_axes=True, 
) 
c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [20]:
output_chart(c)

<IPython.core.display.JSON object>